# Exploring the collation of Calpurnius Flaccus

In this notebook, I present an interactive method to manipulate the collation of the *Declamations* of Calpurnius Flaccus obtained with [CollateX](collatex.net).

## Understanding the data

First it is necessary to understand the structure of the collation data. [Here](http://www.w3resource.com/JSON/structures.php) you can find the generic description of json structures. An object (dictionary) is made of one or more pairs of name/value. The relation name/value is similar to the relation between a word and its definition in a dictionary. An array (list) is an ordered list of values. A json *object* corresponds to a python *dictionary*, and a json *array* to a python *list*. Lists can be combined: a table is a list of lists.

There are two objects in the json collation, which are converted into their equivalent in python: 
* a list of witnesses ("witnesses")
* a table with the aligned text versions ("table")

The list of witnesses works as a header for the table, which can be itself described as:
* a table is a list of rows
* a row is a list of cells
* a cell is a list of tokens
* a token is a dictionary with the following entries:
    * **t : exact word as it appears in the witness**
    * n : optional normalized version of the word
    * locus : optional exact location of the word in the manuscript/edition. Folio/page number, followed by line number
    * note : optional comment 
    * decl: optional declamation number
    * link : optional link to the page of a digital facsimile, where the token appears
    
The order of the rows is important, because it follows the order of the text. Row 0 has the first word(s) of the text, whereas the last row has also the last word(s) of the text. The row's number is called ID number, and will be used later for variant location identification.

## Summary <a name="ToC"></a>
1. [Import Data](#part1).
The first step is to import the collation results into python. 

2. [Functions](#part2).
Then we will create functions to filter the data according to our need. For instance, we may want to find all the unique variants of one witness. Or we would like to see the places where one group of witnesses agrees with each other against another group of witnesses. Some functions will allow us to diplay only a section of the collation, or information specific to one variant location. Including:
  1. [find agreements](#func2-4)
  2. [table to html](#func2-6)
  3. [move tokens](#func2-10)
  4. [add/delete note](#func2-11)
  5. [search](#func2-12)
  6. [save](#func2-13)

3. [Exploration of collation](#part3).
Finally, we will use our functions to explore the collation of Calpurnius Flaccus, with interactive [widgets](http://ipywidgets.readthedocs.io/en/latest/index.html):
  1. [Modify](#part3-1) collation (move tokens, add/del rows, add/del notes, save the new json)
  2. [Find agreements](#part3-2) between witnesses, and save the result into a more complex html file
  3. [Search](#part3-3) the collation
  4. [Clarify](#part3-4) a reading

## Notebook re-use
Any JSON output from collateX, with **tokens "t"**. The other elements are optional.

Some transformations may be needed. For instance:
 * In collation [import data](#import1-2) :  change the base base text. Otherwise, suppress it altogether from the two "table_to_html" [functions](#func2-6) and change the description of the "save_to_html" [function](#func2-13).
 * In the HTML template, title and credits should be updated. The transformation into complex HTML could also be adapted, according to the json model of the new user.

## Possible Improvements
- large amounts of modifications (moving many tokens at the same time)
- find and delete empty rows automatically
- edit notes directly
- save file with popup window
- etc.

## IMPORT DATA <a name="part1"></a>

### Python modules import

In [ ]:
#python modules
import json
import sys
import re
from IPython.display import display,HTML

#ipywidgets modules
from __future__ import print_function
from ipywidgets import *
import ipywidgets as widgets

### Collation import <a name='import1-2'></a>

In [ ]:
#path to the json results of the collation
path = 'json-collations/calpurnius-collation-joint-BCMNPH.json'
#path = 'json-collations/calpurnius-collation-joint-BCMNPH-corr.json'

#open the file
with open (path, encoding='utf-8') as jsonfile:
    #transform the json structure (arrays, objects) into python structure (lists, dictionaries)
    data = json.load(jsonfile)

#list of witnesses
witnesses = data["witnesses"]
print(witnesses)

#table of the aligned text versions
collation = data["table"]

#base text: choose a witness which variants are considered true readings (in green)
#For Calpurnius, the most recent edition of Hakanson is used as the base text
base_text = 'LH'

#the index of a witness is its position in the witness list:
#for instance B1 has position 0, and P1594 has position 9.

## FUNCTIONS <a name="part2"></a> [↑](#ToC)

### Transform a cell c (list of tokens) into a string of text

In [ ]:
#original text
def cell_to_string(c):    
    #tokens t are joined together, separated by a space
    string = ' '.join(token['t'] for token in c)
    return string

#text with normalized tokens
def cell_to_string_norm(c):    
    string = ''
    #word division is not taken into account when comparing the normalized text
    #for this reason we do not add a space in between tokens
    for token in c:
        if 'n' in token:
            string += token['n']
        elif 't' in token:
            string += token['t']
    return string


### Compare cells

In [ ]:
#compare two cells, original text
def compare_cell(c1,c2):
    return cell_to_string(c1) == cell_to_string(c2)

#compare two cells, normalized text
def compare_cell_norm(c1,c2):
    return cell_to_string_norm(c1) == cell_to_string_norm(c2)    

#compare a list of cells, original text
#return true if all the cells are equivalent (they contain the same string of tokens)    
def compare_multiple_cell(cell_list):
    #compare each cell to the next
    for c1,c2 in zip(cell_list, cell_list[1:]):
        if compare_cell(c1,c2) is False:
            comparison = False
            break
    else:
        comparison = True
    return comparison

#compare a list of cells, normalized text
#return true if all the cells are equivalent (they contain the same string of tokens)
def compare_multiple_cell_norm(cell_list):
    #compare each cell to the next
    for c1,c2 in zip(cell_list, cell_list[1:]):
        if compare_cell_norm(c1,c2) is False:
            comparison = False
            break
    else:
        comparison = True
    return comparison

### Find agreements 1

In [ ]:
#this function returns rows of the collation table (table) where a list of witnesses (witlist) agree together.
#we display only variant locations, and not places where all witnesses agree.

def find_agreements(table, witlist):
    result_table = []
    
    #transform widget tuple into actual list
    witlist = list(witlist)
    
    #transform the witnesses names (sigla) into indexes
    witindex = [witnesses.index(wit) for wit in witlist]
    nonwitindex = [witnesses.index(wit) for wit in witnesses if wit not in witlist]

    for row in table:
        #get list of cell for the x witnesses
        cell_list = [row[i] for i in witindex]
        #there must be agreement of the x witnesses (normalised tokens)
        if compare_multiple_cell_norm(cell_list) is True:
            for i in nonwitindex:
                #if they disagree with at least one of the others
                if compare_cell_norm(row[witindex[0]],row[i]) is False:
                    #add row
                    result_table.append(row)
                    #and go to next row
                    break

    return result_table

### Find agreements 2  <a name='func2-4'></a> [↑](#ToC)

In [ ]:
#This function is similar to the previous one:
#it returns rows of the collation table (table) where a list of witnesses (witlist) agree together, but
#against witnesses in a second list (nonwitlist).

#By default, it will return the agreement against all the other witnesses.

def compare_witnesses(table, witlist, nonwitlist=[]):
    result_table = []
    
    #first list of x witnesses which agree together
    witindex = [witnesses.index(wit) for wit in witlist]
    #against all the other witnesses
    if not nonwitlist:
        nonwitindex = [witnesses.index(wit) for wit in witnesses if wit not in witlist]
    #except if a second list of y witnesses is specified
    else:
        nonwitindex = [witnesses.index(wit) for wit in nonwitlist]
    
    #go through the collation table, row by row
    #to find places where the x witnesses agree together against others
    for row in table:
        #get list of cell for the x witnesses
        cell_list = [row[i] for i in witindex]
        #there must be agreement of the x witnesses (normalised tokens)
        if compare_multiple_cell_norm(cell_list) is True:
            for i in nonwitindex:
                #if they agree with one of the other witnesses
                if compare_cell_norm(row[witindex[0]],row[i]) is True:
                    #go to next row
                    break
            else:
                result_table.append(row)
    return result_table

### Find all variants in the collation table

In [ ]:
def view_variants(table):
    result_table = []
    for row in table:
        # there must be at least one variant
        if compare_multiple_cell_norm(row) is False:
            result_table.append(row)       
    return result_table

### Transform the result table into an html table  <a name='func2-6'></a> [↑](#ToC)

In [ ]:
#this is a minimal html, to display in the notebook.

def table_to_html(collation,table):
    
    #table in an html format
    html_table = ''
    #div is for a better slides view. For notebook use, comment it out
    #html_table += '<div style="overflow: scroll; width:960; height:417px; word-break: break-all;">'
    html_table += '<table border="1" style="width: 100%; border: 1px solid #000000; border-collapse: collapse;" cellpadding="4">'
    
    #add a header to the table with columns, one for each witnesses and one for the row ID
    html_table += '<tr>'
    #a column for each witness
    for wit in witnesses:
        html_table += '<th>'+wit+'</th>'
    #optional: column for the declamation number
    #html_table += '<th>Decl</th>'
    #column for the row id
    html_table += '<th>ID</th>'
    html_table += '</tr>'
    
    for row in table:
        #add a row to the html table
        html_table += '<tr>'
        #optional : a variable to store the declamation number (will not be defined in empty rows)
        #declamation = 0
        #fill row with cell for each witness
        for cell in row:
            #transform the token t into a string.
            #we display the original token, not the normalized form
            token = cell_to_string(cell)
            
            #some cells are empty. Thus the declamation number is only available in cell with at least 1 token
            #if len(cell)>0:
            #    declamation = str(cell[0]['decl'])
            
            #if the normalized token is different from the "true reading"
            #it is diplayed in a red cell
            bg = "ffb1b1"
            #otherwise in a green cell
            if compare_cell_norm(cell,row[witnesses.index(base_text)]):
                bg = "d9ead3"
            html_table += '<td bgcolor="'+bg+'">'+token+'</td>'
        
        #optional: add declamation number  
        #html_table += '<td>'+str(location)+'</td>'
        
        #add row ID
        html_table += '<td>'+str(collation.index(row))+'</td>' 
        
        #close the row
        html_table += '</tr>'
    
    #close the table
    html_table += '</table>'
    #html_table += '</div>'

    return html_table 
    

In [ ]:
#this a fancier HTML, but can't be displayed in the notebook (yet)
def table_to_html_fancy(collation,table):
    #table in an html format
    html_table = '<table>'
    
    #add a header to the table with columns
    html_table += '<thead><tr>'
    #a column for each witness
    for wit in witnesses:
        html_table += '<th>'+'<p>'+wit+'</p>'+'</th>'
    #a column for the row id
    html_table += '<th><p>ID</p></th>'
    #close header
    html_table += '</tr></thead><tbody>'
    
    for row in table:
        #add a row to the html table
        html_table += '<tr>'
        for cell in row:
            #transform the token t into a string (original token) 
            token = cell_to_string(cell)
            #if the normalized token is different from the supposed "true reading"
            #it is diplayed in a cell with orange left border
            cl = "orange"
            #otherwise with green left border
            if compare_cell_norm(cell,row[witnesses.index(base_text)]):
                cl = "green"
            #add token to the table, in a text paragraph    
            html_table += '<td class="'+cl+'">'+'<p>'+token
            
            #if there is a note to display, add a little 'i' to indicate there is more hidden information
            for t in cell:
                #in the cell, if we find a token with a note
                if 'note' in t:
                    #add info indicator
                    html_table += ' <a href="#" class="expander right"><i class="fa fa-info-circle"></i></a>'
                    #then stop (even if there are several notes, we display only one indicator)
                    break

            #close the text paragraph in the cell
            html_table += '</p>'
            
            #add paragraphs for hidden content (notes. Not limited to notes only: normalized form could be added, etc.)
            for t in cell:
                if 'note' in t:
                    html_table += '<p class="expandable hidden more-info">Note: '+t['note']+'.</p>'
            
            #when the cell is not empty, add hidden info of page/line numbers. Adapted to make 'locus' optional
            if len(cell)>0 and 'locus' in cell[0]:
            #if len(cell)>0 :
                #add link to images when possible
                if 'link' in cell[0]:
                    url = cell[0]['link']
                    html_table += '<p class="expandable-row hidden more-info"><a target="blank" href='+url+'>'+cell[0]['locus']+'</a></p>'
                else:
                    html_table += '<p class="expandable-row hidden more-info">'+cell[0]['locus']+'</p>'
            
            #close cell
            html_table += '</td>'
        
        #add row ID with indicator of hidden content
        html_table += '<td>'+'<p>'+str(collation.index(row))+' <a href="#" class="expander-row right"><i class="fa fa-ellipsis-v"></i></a></p>'+'</td>' 
        #close the row
        html_table += '</tr>'
    #close the table
    html_table += '</tbody></table>'
    
    return html_table

### Print collation in a text orientation
To read a short passage quickly. The collation table is reversed so that each witness is displayed on a row instead of a column. 

In [ ]:
def print_witnesses_text(table):
    reverse_table = [[row[i] for row in table] for i in range(len(witnesses))]
    for index,row in enumerate(reverse_table):
        text = ''
        for cell in row:
            #the row starts and ends with a token, not a space
            if row.index(cell) == 0 or text == '' or not cell:
                text += cell_to_string(cell)
            #if it is not the start of the string or an empty cell, add a space to separate tokens
            else:
                text += ' '+cell_to_string(cell)
        text += ', '+str(witnesses[index])
        print(text)
    #return reverse_table

### Print information about a reading

In [ ]:
def print_info(rowID, wit):
    #if there is no token
    if len(collation[int(rowID)][witnesses.index(wit)]) == 0:
            print('-')
    else:
        for token in collation[int(rowID)][witnesses.index(wit)]:
            print(', '.join(token[feature] for feature in token))

### Get the ID of a row

In [ ]:
def get_pos(row):
    return collation.index(row)

### Move a token to the previous (up) or next (down) row  <a name='func2-10'></a> [↑](#ToC)

In [ ]:
def move_token_up(rowID, wit):
    try:
        #select the first token
        token = collation[rowID][witnesses.index(wit)].pop(0)
        #append it at the end of the cell in the previous row
        collation[rowID-1][witnesses.index(wit)].append(token)
    except:
        print("There is no token to move.")

def move_token_down(rowID, wit):
    try:
        #select the last token
        token = collation[rowID][witnesses.index(wit)].pop()
        #add it at the beginning of the cell in the next row
        collation[rowID+1][witnesses.index(wit)].insert(0, token)
    except:
        print("There is no token to move.")

### Add / delete a row

In [ ]:
def add_row_after(rowID):
    #rowID must be within collation table
    if rowID < 0 or rowID > len(collation):
        print('Row'+rowID+' does not exist.')
    else:
        #create an empty row
        new_row = []
        #for each witness in the collation
        for wit in witnesses:
            #add an empty list of tokens to the row
            new_row.append([])
        #insert new row in the collation, after the row passed in argument (+1)
        collation.insert(rowID+1, new_row)
        print('Row added!')
    
def delete_row(rowID):
    #rowID must be within collation table
    if rowID < 0 or rowID > len(collation):
        print('Row'+rowID+' does not exist.')
    else:
        collation.pop(rowID)
        print('Row deleted!')


### Add / Delete a note  <a name='func2-11'></a> [↑](#ToC)

In [ ]:
#add or modify a note
def add_note(wit, rowID, token, note):
    #for each token in the witnes/row selected
    for t in collation[rowID][witnesses.index(wit)]:
        #check that the note isnt empty and that the token corresponds to the one passed by user
        if note is not '' and t['t'] == token:
            if 'note' in t:
                #add comment to already existing note
                t['note'] += ' '+note
            else:
                #or create a new note
                t['note'] = note
        else:
            print('Note could not be added.\nCheck that your note is not empty, and the token is correct')

#delete completely a token's note            
def del_note(wit, rowID, token):
    #for each token in the witnes/row selected
    for t in collation[rowID][witnesses.index(wit)]:
        #check that token corresponds and that note exists
        if t['t'] == token and 'note' in t:
            #delete note    
            t.pop('note')
        else:
            #error message
            print('There is no note to delete')

### Search  <a name='func2-12'></a> [↑](#ToC)

In [ ]:
def search(table,text):
    #result table to build
    result_table = []
    
    #go through each row of the collation table
    for row in table:
        #go through each cell
        for cell in row:
            #if the cell text matches the search text
            if text in cell_to_string_norm(cell) or text in cell_to_string(cell):
                #add row to the result table
                result_table.append(row)
                #go to next row
                break
    
    #if the result table is empty, the text was not found in the collation
    if result_table == []:
        print(text+" was not found!")
    
    return result_table

### Save results  <a name='func2-13'></a> [↑](#ToC)

In [ ]:
#save the json file with update in the collation table
def save_json(path, table):
    #combine new collation table with witnesses, so as to have one data variable
    data = {'witnesses':witnesses, 'table':table}
    #open a file according to path
    with open(path, 'w') as outfile:
        #write the data in json format
        json.dump(data, outfile)

In [ ]:
#save a subset of the collation table into fancy HTML version, with a small text description
def save_table(descr, table, path):
    #path to template
    template_path = 'alignment-tables/template.html'
    
    #load the text of the template into a variable html
    with open(template_path, 'r', encoding='utf-8') as infile:
        html = infile.read()
    
    #add base text to description
    if base_text:
        descr += '<br>Agreement with the base text '+base_text+' is marked in green.'
        descr += ' Variation from '+base_text+' is marked in red.' 
        
    #modify template: replace the comment with descr paragraph
    html = re.sub(r'<!--descr-->',descr,html)
    
    #modify template: replace the comment with table
    html = re.sub(r'<!--table-->',table,html)
    
    #save
    with open(path, 'w', encoding='utf-8') as outfile:
        outfile.write(html)


## INTERACTIVE EXPLORATION OF THE COLLATION TABLE <a name="part3"></a> [↑](#ToC)

### Update the collation results <a name='part3-1'></a> [↑](#ToC)
In this section, we will see how to view a selection of the collation table and update it.

Possible updates:
  1. Add/delete a row
  3. Move one token up/down
  4. Add notes
  3. Save

**Note** : it is only possible to move the last token (or word) in a cell down to the first place in the next cell; or vice-versa to move the first token of a cell up to the last place in the previous cell. This is to prevent any change in the word order of a witness, and *to keep the correct text sequence*.

In [ ]:
#an exctract of the collation table selected with interactive widgets

#widget for HTML display
w1_html = widgets.HTML(value="")

#define the beginning of extract
w_from = widgets.BoundedIntText(
    value=6,
    min=0,
    max=len(collation)-1,
    description='From:',
)

#define the end of extract
#because of python list slicing, the last number is not included in the result.
#to make it more intuitive, the "to" number is added +1 in collation_extract function
w_to = widgets.BoundedIntText(
    value=11,
    min=0,
    max=len(collation)-1,
    description='To:',
)

#binding widgets with table_to_html function
def collation_extract(a, b):
    widget.result = ""
    x = a
    y = b+1
    w1_html.value = table_to_html(collation,collation[x:y])

#interactive selection of collation table extract
interact(collation_extract, a=w_from, b=w_to)
#display selection widget (6-11)
display(w1_html)

In [ ]:
#Widgets for:
#move tokens up/down
#add/delete rows
#add/delete notes to a specific token

#widget to select a witness        
w_wit = widgets.Dropdown(
    options = witnesses,
    value = 'LH',
    description = 'Witness:',
)
#widget to select a row
w_rowID = widgets.BoundedIntText(
    value=9,
    min=0,
    max=len(collation),
    description='ID:',
)
#widget to select a specific token
w_token = widgets.Text(
    description = 'Token:',
    value = 'proximi'
)
#widget to enter text note
w_note = widgets.Text(
    description = 'Note:',
)

In [ ]:
#add row after
interact(add_row_after, rowID=w_rowID, __manual=True)

In [ ]:
#delete row
interact(delete_row, rowID=w_rowID, __manual=True)

In [ ]:
#move token down
interact(move_token_down, rowID=w_rowID, wit=w_wit, __manual=True)

In [ ]:
#move token up
interact(move_token_up, rowID=w_rowID, wit=w_wit, __manual=True)

In [ ]:
#add/delete notes
#link add button and function
def add_on_click(sender):
    add_note(wit=w_wit.value, rowID=w_rowID.value, token=w_token.value, note=w_note.value)
    #check result
    print('Result:')
    print_info(w_rowID.value, w_wit.value)
    print('\n')

#add a note button
w_add_note = widgets.Button(description='Add note', button_style='success')   
w_add_note.on_click(add_on_click)

#link del button and function
def del_on_click(b):
    del_note(wit=w_wit.value, rowID=w_rowID.value, token=w_token.value)
    #check result
    print('Result:')
    print_info(w_rowID.value, w_wit.value)

#delete a note button
w_del_note = widgets.Button(description='Delete note', button_style='danger')
w_del_note.on_click(del_on_click)

#dislpay widgets
display(w_wit, w_rowID, w_token, w_note)
display(w_add_note, w_del_note)

In [ ]:
#save new json

#path to the new file
path_new_json = 'json-collations/calpurnius-collation-joint-BCMNPH-corr.json'

#save button to click
w_button = widgets.Button(description="Save JSON", button_style='info')

#on click
def on_button_clicked(b):
    #save json of the whole collation
    save_json(path_new_json, collation)

#link btw button and onclick function 
w_button.on_click(on_button_clicked)

#save json
display(w_button)

### Find agreements of witnesses against others <a name='part3-2'></a> [↑](#ToC)

In the first drop-down menu, you will be able to choose one or more manuscript which agree together in places where all the other witnesses have a different reading. It means that if you pick only one witness you will see it's **unique variants**.

On the other hand, you may want to see the **agreements of witnesses against another group of witnesses** selected in the second drop-down menu. This allows you for instance to ignore modern editors from the comparison. This option also allows to compare for groups of manuscripts to see if they share erroneous readings (or innovations), for instance.

Finally, if you chose only one witness in each dropdown menu, you will be able to see the **differences between the two witnesses**.

In [ ]:
#widget for HTML display
w2_html = widgets.HTML(value="")

#selection of a group of witnesses which share the same readings
w1 = widgets.SelectMultiple(
    description="Agreements:",
    options=witnesses
)

#selection of a secong group of witnesses
w2 = widgets.SelectMultiple(
    description="Against:",
    options=witnesses
)

def collation_compare(table, a, b):
    #transform widget tuple into actual list
    if isinstance(a, (tuple)):
        witlist = list(a)
        nonwitlist = list(b)
    else:
        witlist = [a]
        nonwitlist = [b]
    if not a:
        print("No witness selected.")
    else:
        #set HTML display value
        w2_html.value = table_to_html(table,compare_witnesses(table, witlist, nonwitlist))

#-----------
#save button
w_save = widgets.Button(description="Save Table", button_style='info')

#description of the table
w_descr = widgets.Text(value="Table description")

#path for result
path_result = 'alignment-tables/collation.html'

def on_button_clicked(x):
    #transform widget tuple into actual list
    if isinstance(w1.value, (tuple)):
        witlist = list(w1.value)
        nonwitlist = list(w2.value)
    else:
        witlist = [w1.value]
        nonwitlist = [w2.value]
    if not w1.value:
        print("No table to save.")
    else:
        descr = str(w_descr.value)
        table = table_to_html_fancy(collation,compare_witnesses(collation, witlist, nonwitlist))
        save_table(descr, table, path_result)

#link button with saving action
w_save.on_click(on_button_clicked)
#---------------

#find agreements between witnesses or unique readings
interact(collation_compare, table=fixed(collation), a=w1, b=w2)

display(w2_html)
display(w_descr)
display(w_save)

### Search the collation <a name='part3-3'></a> [↑](#ToC)
Basic search. It will check for token t or normalized form n, and return rows where there is at least one match. 

In [ ]:
#widget for HTML display
w3_html = widgets.HTML(value="")

#do the search
def search_collation(table,text):
    w3_html.value = table_to_html(table,search(table,text))
    
#search collation with interactive text input
interact(search_collation, table=fixed(collation),text="calpurnius",__manual=True)
display(w3_html)

### Clarify one reading <a name='part3-4'></a> [↑](#ToC)

In [ ]:
#459/C1, 932/M1, 9/LH
interact(print_info, rowID=w_rowID, wit=w_wit)